# Homework 3: N-Gram Language Models

## Total Points: 95 + 5 bonus
- **Overview**: In the textbook, language modeling was defined as the task of predicting the next word in a sequence given the previous words. In this assignment, we will focus on the related problem of predicting the next character in a sequence given the previous characters.

- **Learning goals**:
    - Understand how to compute language model probabilities using maximum likelihood estimation.
    - Implement basic smoothing, back-off and interpolation.
    - Have fun using a language model to probabilistically generate texts.
    - Use a set of language models to perform text classification.

- **Data**: We will provide you with training and development data that has been manually labeled. We will also give you a test set without labels. You will build a classifier to predict the labels on our test set. You can upload your classifier’s predictions to Gradescope. We will score its predictions and maintain a leaderboard showing whose classifier has the best performance.

- **Delieverables:** This assignment has several deliverables:
    - Your implementations for the functions in the skeleton code (this notebook)
    - Your model’s output for the test set (your model will be ranked on a leaderboard against the other students’ outputs)

- **Grading**: We will use the auto-grading system called `PennGrader`. To complete the homework assignment, you should implement anything marked with `#TODO` and run the cell with `#PennGrader` note. **There will be no hidden tests in this assignment.** In other words, you will know your score once you finish all the `#TODO` and run all the `#PennGrader` tests!


## Recommended Readings
- [Language Modeling with N-grams](https://web.stanford.edu/~jurafsky/slp3/3.pdf). Dan Jurafsky and James H. Martin. Speech and Language Processing (3rd edition draft) .
- [A Bit of Progress in Language Modeling](https://arxiv.org/abs/cs/0108005). Joshua Goodman. Computer Speech and Language .
- [The Unreasonable Effectiveness of Character-level Language Models](http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139). Yoav Goldberg. Response to Andrej Karpathy's blog post. 2015.
- [Language Independent Authorship Attribution using Character Level Language Models](http://www.aclweb.org/anthology/E/E03/E03-1053.pdf). Fuchun Pen, Dale Schuurmans, Vlado Keselj, Shaojun Wan. EACL 2003.

## To get started, **make a copy** of this colab notebook into your google drive!

## Setup 1: PennGrader Setup [4 points]

In [54]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install penngrader-client

In [55]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Overwriting notebook-config.yaml


In [56]:
!cat notebook-config.yaml


grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'


In [57]:
from penngrader.grader import *

## TODO - Start
STUDENT_ID = 56803282 # YOUR PENN-ID GOES HERE AS AN INTEGER#
## TODO - End

SECRET = STUDENT_ID
grader = PennGrader('notebook-config.yaml', 'CIS5300_OL_23Su_HW3', STUDENT_ID, SECRET)

PennGrader initialized with Student ID: 56803282

Make sure this correct or we will not be able to store your grade


In [58]:
# check if the PennGrader is set up correctly
# do not chance this cell, see if you get 4/4!
name_str = 'Jacky Choi'
grader.grade(test_case_id = 'name_test', answer = name_str)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Setup 2: Dataset / Packages
- **Run the following cells without changing anything!**
- [Loading dataset from huggingface](https://huggingface.co/docs/datasets/v1.8.0/loading_datasets.html#from-local-files)

In [59]:
import math, random
from collections import Counter
from dill.source import getsource

In [60]:
## DO NOT CHANGE ANYTHING, JUST RUN

def get_class_source(cls):
    import re
    class_name = cls.__name__
    from IPython import get_ipython
    ipython = get_ipython()
    inputs = ipython.user_ns['In']
    pattern = re.compile(r'^\s*class\s+{}\b'.format(class_name))
    for cell in reversed(inputs):
        if pattern.search(cell):
            return cell
    return None

In [61]:
!gdown 19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T # https://drive.google.com/file/d/19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T/view?usp=share_link
!unzip -o cities_train.zip
!gdown 1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu # https://drive.google.com/file/d/1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu/view?usp=sharing
!unzip -o cities_val.zip
!gdown 1mz6EdH-4pOwr-o3lHDlh1SS7oRdydNr1 # https://drive.google.com/file/d/1mz6EdH-4pOwr-o3lHDlh1SS7oRdydNr1/view?usp=sharing
!gdown 11es6r9fVz_wf_Q2T7oP2IQDkAAmhtzf4 # https://drive.google.com/file/d/11es6r9fVz_wf_Q2T7oP2IQDkAAmhtzf4/view?usp=share_link
!gdown 19P37gR_XDup-h3a4LdKtQWbdM-lGPqin # https://drive.google.com/file/d/19P37gR_XDup-h3a4LdKtQWbdM-lGPqin/view?usp=sharing
!gdown 1kKfh2oaxinBPVwWaux6FA9TerqAH5nHw # https://drive.google.com/file/d/1kKfh2oaxinBPVwWaux6FA9TerqAH5nHw/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=19ZFc_nFF3T-LTI9rxXUy1XyMaoGSzN3T
To: /content/cities_train.zip
100% 160k/160k [00:00<00:00, 24.7MB/s]
Archive:  cities_train.zip
  inflating: train/af.txt            
  inflating: train/de.txt            
  inflating: train/fi.txt            
  inflating: train/fr.txt            
  inflating: train/in.txt            
  inflating: train/ir.txt            
  inflating: train/cn.txt            
  inflating: train/za.txt            
  inflating: train/pk.txt            
Downloading...
From: https://drive.google.com/uc?id=1BH9qhvR7POqmGPOPadqADv-NxdGBgyzu
To: /content/cities_val.zip
100% 7.56k/7.56k [00:00<00:00, 21.6MB/s]
Archive:  cities_val.zip
  inflating: val/af.txt              
  inflating: val/cn.txt              
  inflating: val/de.txt              
  inflating: val/fi.txt              
  inflating: val/fr.txt              
  inflating: val/ir.txt              
  inflating: val/za.txt              
  inflating: val/pk.txt       

# Section 0: Generating N-Grams [5 points]
- **Problem 0:** Write a function `ngrams(n, text)` that produces a list of all n-grams of the specified size from the input text. Each n-gram should consist of a 2-element tuple `(context, char)`, where the context is itself an n-length string comprised of the n characters preceding the current character. The sentence should be padded with n ~ characters at the beginning (we’ve provided you with `start_pad(n)` for this purpose). If n=0 , all contexts should be empty strings. You may assume that n≥0.

```
>>> ngrams(1, 'abc')
[('~', 'a'), ('a', 'b'), ('b', 'c')]

>>> ngrams(2, 'abc')
[('~~', 'a'), ('~a', 'b'), ('ab', 'c')]
```

In [62]:
### DO NOT CHANGE ###
def start_pad(n):
    ''' Returns a padding string of length n to append to the front of text
        as a pre-processing step to building n-grams '''
    return '~' * n
### DO NOT CHANGE ###

In [63]:
def ngrams(n, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''

    ## AUTOGRADER HELPER ##
    def start_pad(n):
        ''' Returns a padding string of length n to append to the front of text
            as a pre-processing step to building n-grams '''
        return '~' * n
    ## AUTOGRADER HELPER ##



    ## TODO: create n-grams given the text
    #>>> ngrams(1, 'abc')
    #[('~', 'a'), ('a', 'b'), ('b', 'c')]
    # >>> ngrams(2, 'abc')
    # [('~~', 'a'), ('~a', 'b'), ('ab', 'c')]
    ngrams_arr = []
    padding = start_pad(n) + text

    for i in range(len(text)):
        context = padding[i:i+n]
        char = text[i]
        ngrams_arr.append((context, char))

    return ngrams_arr



In [64]:
print(ngrams(1, 'abc'))
print(ngrams(2, 'abc'))

[('~', 'a'), ('a', 'b'), ('b', 'c')]
[('~~', 'a'), ('~a', 'b'), ('ab', 'c')]


In [65]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q0_ngrams', answer = getsource(ngrams))

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 1: Creating an N-Gram Model [36 points]
In this section, you will build a simple n-gram language model that can be used to generate random text resembling a source document. Your use of external code should be limited to built-in Python modules, which excludes, for example, `NumPy` and `NLTK`.


In [103]:
class NgramModel(object):
    ''' A basic n-gram model using add-k smoothing '''

    def __init__(self, n, k = 0):
        # About `k`: you don't need to worry about it for problem 1.1, it is something you will need in problem 2.1
        self.k = k
        # contexts is a dictionary with the context as keys and corresponding value is a dictionary of count and
        # chars which is a dictionary of characters as key and it's count as value.
        # Please do not change the format of storing the counts and characters, the autograder uses this structure
        # For example:
        # if the context is 'ab' with count=3 and characters following it are 'a' and 'b' with counts 2 and 1 respectively
        # then the context dictionary would look like this:
        # {
        #   'ab' : {'count': 3,'chars': {'a' : 2, 'b' : 1}},
        #   'aa' : {...},
        #   ...
        # }
        self.contexts = {}

        ## TODO 1.1 - see problem 1.1
        self.n = n
        self.vocab = set()

    def get_vocab(self):
        ''' Returns the set of characters in the vocab '''
        ## TODO 1.1 - see problem 1.1
        return self.vocab

    def get_context(self):
        ''' Returns the dictionary for context '''
        return self.contexts


    '''Write a method update(self, text) which computes the n-grams for the input sentence and updates the internal
    counts. Also write a method prob(self, context, char) which accepts an n-length string representing a context and
    a character, and returns the probability of that character occuring, given the preceding context. If you encounter
    a novel context, the probability of any given char should be 1/V where V is the size of the vocab.'''
    def update(self, text):
        ''' Updates the model n-grams based on text '''
        ## TODO 1.2 - see problem 1.2
        ngrams_arr = ngrams(self.n, text) #full list of ngram tuples(context, char)
        for context, char in ngrams_arr:
            if context not in self.contexts:
                self.contexts[context] = {'count': 0, 'chars': {}}
            #increment count of countext by 1
            self.contexts[context]['count'] += 1
            #captures char after context
            if char not in self.contexts[context]['chars']:
                self.contexts[context]['chars'][char] = 0
            #increment this char by 1
            self.contexts[context]['chars'][char] += 1
            #add to the vocab set
            self.vocab.add(char)

    def prob(self, context, char):
        ''' Returns the probability of char appearing after context '''

        if self.k == 0:

            if context in self.contexts:
                if char in self.contexts[context]['chars']:
                    return self.contexts[context]['chars'][char] / self.contexts[context]['count']
                else:
                    return 0
            else:
                return 1 / len(self.vocab) if len(self.vocab) > 0 else 0
        else:
        # TODO: Implement add-k smoothing here (for self.k > 0)

            if context in self.contexts:
                numerator = self.contexts[context]['chars'].get(char, 0) + self.k
                denominator = self.contexts[context]['count'] + (self.k * len(self.vocab))
                return numerator / denominator
            else:

                return self.k / (self.k * len(self.vocab)) if len(self.vocab) > 0 else 0


    def random_char(self, context):
        ''' Returns a random character based on the given context and the
            n-grams learned by this model '''
        ## TODO 1.3 - see problem 1.3
        #Write a method random_char(self, context) which returns a random character according to the probability distribution determined by the given context. Specifically, let  𝑉=⟨𝑣1,𝑣2,⋯,𝑣𝑛⟩  be the vocab, sorted according to Python's natural lexicographic ordering, and let  𝑟  be a random number between 0 and 1. Your method should return the character  𝑣𝑖  such that:
        r = random.random() #0 - 1
        prev_sum = 0
        for i, char in enumerate(sorted(self.vocab)):
            post_sum = prev_sum + self.prob(context, char)
            if prev_sum < r < post_sum:
                return char
            prev_sum = post_sum



    def random_text(self, length):
        ''' Returns text of the specified character length based on the
            n-grams learned by this model '''
        ## TODO 1.4 - see problem 1.4
        context = start_pad(self.n)
        text = ''
        for _ in range(length):
            char = self.random_char(context)
            text += char
            context = context[1:] + char
        return text

    def perplexity(self, text):
        ''' Returns the perplexity of text based on the n-grams learned by
            this model '''
        ## TODO 2.2 - see problem 2.2

        length_text = len(text)
        log_prob = 0
        ngrams_arr = ngrams(self.n, text)
        for context, char in ngrams_arr:
           prob = self.prob(context,char)
           if prob == 0:
              return float('inf')
           log_prob += math.log(prob)
        perplexity = math.exp(-1/length_text * log_prob)
        return perplexity


## 1.1 Initialization [3 points]
In the `NgramModel` class, write an initialization method `__init__(self, n, k)` which stores the order n of the model and initializes any necessary internal variables. Then write a method `get_vocab(self)` that returns the vocab (this is the `set` of all characters used by this model).

- **Problem 1.1:** finish `__init__(self, n, k)` and `get_vocab(self)` [3 points]

In [104]:
# PennGrader - DO NOT CHANGE
ngram_model = NgramModel(1, 0)
grader.grade(test_case_id = 'test_q11_init_get_vocab', answer = get_class_source(NgramModel))

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.2 Update and Calculate Probabilities [15 points]
Write a method `update(self, text)` which computes the n-grams for the input sentence and updates the internal counts. Also write a method `prob(self, context, char)` which accepts an n-length string representing a context and a character, and returns the probability of that character occuring, given the preceding context. If you encounter a novel `context`, the probability of any given `char` should be 1/V where V is the size of the vocab.

- **Problem 1.2:** finish `update(self, text)` and `prob(self, context, char)` [15 points]

In [68]:
# PennGrader - DO NOT CHANGE
all_test_cases = [
    (1, ['abab', 'abcd'], [('b','c'), ('a', '~')]),
    (2, ['ababa', 'abcdddd', 'babcdabag'], [('ab', 'a'), ('ad', 'b')]),
    (3, ['ababa', 'abcdddd', 'babcdabag'], [('ddd', 'd'), ('~~c', 'f')])
]

all_test_results = []
for test_data in all_test_cases:
    nm = NgramModel(test_data[0], 0)
    for text in test_data[1]:
        nm.update(text)
    vocab = nm.get_vocab()
    probs = []
    for prob_check in test_data[2]:
        prob = nm.prob(prob_check[0], prob_check[1])
        probs.append(prob)
    all_test_results.append((vocab, probs))

# reload_grader()
grader.grade(test_case_id = 'test_q12_update_prob', answer = all_test_results)

Correct! You earned 15/15 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.3 Random Character [6 points]

Write a method `random_char(self, context)` which returns a random character according to the probability distribution determined by the given context. Specifically, let $V=\langle v_1,v_2, \cdots, v_n \rangle$ be the vocab, sorted according to Python's natural lexicographic ordering, and let $r$ be a random number between 0 and 1. Your method should return the character $v_i$ such that:
$$
\sum_{j=1}^{i-1} P(v_j\ |\ \text{context}) \le r < \sum_{j=1}^i P(v_j\ | \ \text{context}).
$$
You should use a single call to the `random.random()` function to generate $r$, for example:
```
>>> m = NgramModel(0, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> random.seed(1)
>>> [m.random_char('') for i in range(25)]
['a', 'c', 'c', 'a', 'b', 'b', 'b', 'c', 'a', 'a', 'c', 'b', 'c', 'a', 'b', 'b', 'a', 'd', 'd', 'a', 'a', 'b', 'd', 'b', 'a']
```

- **Problem 1.3:** finish `random_char(self, context)` [6 points]

In [69]:
# PennGrader - DO NOT CHANGE
corpus = ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab']
nm = NgramModel(1, 0)
for text in corpus:
    nm.update(text)

random_seed_lst = [2, 3, 4]
random_char_lst = []
for seed in random_seed_lst:
    random.seed(seed)
    random_char = [nm.random_char('') for i in range(25)]
    random_char_lst.append(random_char)

grader.grade(test_case_id = 'test_q13_rand_char', answer = random_char_lst)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.4 Random Text [6 points]
In the `NgramModel` class, write a method `random_text(self, length)` which returns a string of characters chosen at random using the `random_char(self, context)` method. Your starting context should always be $n$ ~ characters, and the context should be updated as characters are generated. If $n=0$, your context should always be the empty string. You should continue generating characters until you've produced the specified number of random characters, then return the full string.

```python
>>> m = NgramModel(1, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> random.seed(1)
>>> m.random_text(25)
abcdbabcdabababcdddabcdba
```

- **Problem 1.4:** finish `random_text(self, length)` [6 points]

In [70]:
# PennGrader - DO NOT CHANGE
corpus = ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab']
nm = NgramModel(1, 0)
for text in corpus:
    nm.update(text)

random_seed_lst = [42, 43, 44]
random_text_lst = []
for seed in random_seed_lst:
    random.seed(seed)
    random_text = nm.random_text(25)
    random_text_lst.append(random_text)

grader.grade(test_case_id = 'test_q14_rand_text', answer = random_text_lst)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.5 Writing Shakespeare [6 points]

Now you can train a language model. We have provided this corpus of Shakespeare at `shakespeare_input.txt` (if you cannot find it, re-run the `Setup 2: Dataset / Packages` section).

We’ve also given you the function `create_ngram_model(model_class, path, n, k)` that will create and return an n-gram model trained on the entire file path provided and `create_ngram_model_lines(model_class, path, n, k)` that will create and return an n-gram model trained line-by-line on the file path provided. You should use the first for the Shakespeare file and the second for the city name files.

Try generating some Shakespeare with different order n-gram models. You should try running the following commands:

```python
>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 2)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 3)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4)
>>> m.random_text(250)

>>> m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 7)
>>> m.random_text(250)
```

What do you think? Is it as good as [1000 monkeys working at 1000 typewriters](https://www.youtube.com/watch?v=no_elVGGgW8)?

After generating a bunch of short passages, do you notice anything? *They all start with F!* In fact, after we hit a certain order, the first word is always *First*?  Why is that? Is the model trying to be clever? Is it saying *First, I will generate the word "First"*?  No, probably not.  Explain what is going on in your writeup.

- **Answer 1.5:** Generate Shakespear texts with at least 3 numbers of n and discuss on the results [6 points]

After looking at Shakespeare's input text file, I found that the word First was very common in beginning a dialogue for his writings as well as many names which also appear to be predicted by the models. I believe the ngram model training could have adopted the Shakespearian writing style based on the predictions, patterns and probability that these words would appear. Some of the most common phrases include First ANY_NAME.

In [71]:
## DO NOT CHANGE ##
def create_ngram_model(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        model.update(f.read())
    return model

def create_ngram_model_lines(model_class, path, n=2, k=0):
    ''' Creates and returns a new n-gram model trained on the city names
        found in the path file '''
    model = model_class(n, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        for line in f:
            model.update(line.strip())
    return model

In [72]:
# Example
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 7)
m.random_text(250)

'First Citizen:\nThis is the sad and we been impostor that sets up\nhis head, and Warwick knew it all of good?\n\nOCTAVIUS CAESAR:\nI must not his\nname out such a lose and sudden command,\nIf crooked sadly themselves alone?\n\nLUCIUS:\nMad for the powers? show'

In [73]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 4)
m.random_text(250)

"First Soldier father for a structed: my friences, arity:\nBut, for I will ours\nFrom o' thee word can discorn?\nEither heavens the well as Joan will take, whereign's hire the fear: she's proteus, all I had go:\nCome thus.\n\nRODERICLES:\nThe stress,\nWe five"

In [74]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 5)
m.random_text(250)

"First to watchful stomach a night.\nShould, Petruchio is thrive,\nI wouldst thou are to present thy father guess bravell'd Derceth thee well! what the gentle, knight, good and that\nHath made a sop o' t' oppress'd intervalued throw shall, Kate, what a v"

In [75]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', 6)
m.random_text(250)

"First Senator:\nWe are happiness.\n\nKING CLAUDIO:\nUnhappy.\n\nCORIOLANUS:\nWhat dost not straight;\nFour feast and at lustier task in his loyalty!\nHere comes.\nThat thou see thy bladed ground, an God forbid it.\n\nSIR ANDREW:\nVery man hence, and devour'd,\nBy "

# Section 2: Smoothing, Perplexity, and Interpolation [30 points]

In this part of the assignment, you'll adapt your code in order to implement several of the  techniques described in [Section 3.5 of the Jurafsky and Martin textbook](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

## 2.1 Smoothing [6 points]

Laplace Smoothing is described in section 3.5.1 of the book. Laplace smoothing adds one to each count (hence its alternate name *add-one smoothing*). Since there are *V* characters in the vocabulary and each one was incremented, we also need to adjust the denominator to take into account the extra V observations.

$$P_{Laplace}(w_i) = \frac{count_i + 1}{N+|V|}$$

A variant of Laplace smoothing is called *Add-k smoothing* or *Add-epsilon smoothing*. This is described in section *Add-k 3.5.2*.

```python
>>> m = NgramModel(1, 1)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.prob('a', 'a')
0.14285714285714285
>>> m.prob('a', 'b')
0.5714285714285714
>>> m.prob('c', 'd')
0.4
>>> m.prob('d', 'a')
0.25
```

- **Problem 2.1**: Update your `NgramModel` code from Part 1 to implement add-k smoothing. (see comment of `## TODO 2.1`) [6 points]




In [76]:
# PennGrader - DO NOT CHANGE
smoothing_test_cases = [
    (1, 1, ['abab', 'abcd'], ('b', 'a')),
    (2, 1, ['ababa', 'abcdddd', 'babcdabag'], ('ba', 'b')),
    (3, 2, ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab'], ('aba', 'b'))
]
smoothing_test_results = []
for i, j, corpus, test_words in smoothing_test_cases:
    nm = NgramModel(i, j)
    for text in corpus:
        nm.update(text)
    smoothing_test_results.append(nm.prob(test_words[0], test_words[1]))

grader.grade(test_case_id = 'test_q21_smoothing', answer = smoothing_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 2.2 Perplexity [12 points]

How do we know whether a language model is good? There are two basic approaches:
1. Task-based evaluation (also known as **extrinsic** evaluation), where we use the language model as part of some other task, like automatic speech recognition, or spelling correcktion, or an OCR system that tries to covert a professor's messy handwriting into text.
2. **Intrinsic** evaluation: Intrinsic evaluation tries to directly evalute the goodness of the language model by seeing how well the probability distributions that it estimates are able to explain some previously unseen test set.

Here's what the textbook says:

> For an intrinsic evaluation of a language model we need a test set. As with many of the statistical models in our field, the probabilities of an N-gram model come from the corpus it is trained on, the training set or training corpus. We can then measure the quality of an N-gram model by its performance on some unseen data called the test set or test corpus. We will also sometimes call test sets and other datasets that are not in our training sets held out corpora because we hold them out from the training data.

> So if we are given a corpus of text and want to compare two different N-gram models, we divide the data into training and test sets, train the parameters of both models on the training set, and then compare how well the two trained models fit the test set.

> But what does it mean to "fit the test set"? The answer is simple: whichever model assigns a higher probability to the test set is a better model.

We'll implement the most common method for intrinsic metric of language models: *perplexity*.  The perplexity of a language model on a test set is the inverse probability of the test set, normalized by the number of characters. For a test set $W = w_1 w_2 ... w_N$:

$$Perplexity(W) = P(w_1 w_2 ... w_N)^{-\frac{1}{N}}$$

$$ = \sqrt[N]{\frac{1}{P(w_1 w_2 ... w_N)}}$$

$$ = \sqrt[N]{\prod_{i=1}^{N}{\frac{1}{P(w_i \mid w_1 ... w_{i-1})}}}$$

Now implement the `perplexity(self, text)` function in `NgramModel`. A couple of things to keep in mind:
1. Numeric underflow is going to be a problem, so consider using logs.
2. Perplexity is undefined if the language model assigns any zero probabilities to the test set. In that case your code should return positive infinity - `float('inf')`.
3. On your unsmoothed models, you'll definitely get some zero probabilities for the test set. To test you code, you should try computing perplexity on the training set, and you should compute perplexity for your language models that use smoothing and interpolation.

```python
>>> m = NgramModel(1, 0)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.perplexity('abcd')
1.189207115002721
>>> m.perplexity('abca')
inf
>>> m.perplexity('abcda')
1.515716566510398
```

- **Problem 2.2**: implement the `perplexity(self, text)` function in `NgramModel` [6 points]

In [77]:
# PennGrader - DO NOT CHANGE
ppl_test_cases = [
    (1, 0, ['abab', 'abcd'], 'abca'),
    (2, 0, ['ababa', 'abcdddd', 'babcdabag'], 'ba'),
    (3, 1, ['ababa', 'abcdddd', 'babcdabag', 'dbaa', 'dbab'], 'abcdddddddddd')
]
ppl_test_results = []
for i, j, corpus, test_words in ppl_test_cases:
    nm = NgramModel(i, j)
    for text in corpus:
        nm.update(text)
    ppl_test_results.append(nm.perplexity(test_words))

grader.grade(test_case_id = 'test_q22_ppl', answer = ppl_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Answer 2.2:** Compare and discuss the perplexity for text that is similar and different from Shakespeare's plays. We provide you two dev text files, a New York Times article (`nytimes_article.txt`) and several of Shakespeare's sonnets (`shakespeare_sonnets.txt`); if you cannot find them, re-run `Setup 2: Dataset / Package` at the beginning. Also, feel free to experiment with your own text. [6 points]

I am surprised by the perplexity score because I thought it would be much lower expecially when comparing shakespeare to shakespeare. It was obvious that the nytimes article would be incredibly high because it is a completely different text, but the shakespeare texts were also higher than I thought. I was thinking that it should be around 2-3 instead for the bigram model since there should be very little differences to writing style. The 2+ - gram models had much greater perplexities even when i tried to adjust the smoothing. I learned that perplexity is an extemely strict metric from this. This could be important for generative ai results when we want extremely accurate information.

I also included a comparison to a book called Jack Daniels Running Formula, a book about running and it resulted in an extremely high perplexity score, even higher than nytimes article. At first, I assumed that maybe its because the textbook is much longer in length, but perplexity is usually a normalized length so that cant be the case.

In [78]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', n=3, k=1)
with open("shakespeare_sonnets.txt", encoding='utf-8', errors='ignore') as f:
    text = f.read()
m.perplexity(text)

6.122231996035264

In [79]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', n=3, k=2)
with open("shakespeare_sonnets.txt", encoding='utf-8', errors='ignore') as f:
    text = f.read()
m.perplexity(text)

6.3881481890279685

In [80]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', n=4, k=1)
with open("shakespeare_sonnets.txt", encoding='utf-8', errors='ignore') as f:
    text = f.read()
m.perplexity(text)

6.485626437020774

In [81]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', n=7, k=1)
with open("shakespeare_sonnets.txt", encoding='utf-8', errors='ignore') as f:
    text = f.read()
m.perplexity(text)

21.689731240301278

In [82]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', n=7, k=3)
with open("shakespeare_sonnets.txt", encoding='utf-8', errors='ignore') as f:
    text = f.read()
m.perplexity(text)

29.851819926569483

In [83]:
m = create_ngram_model(NgramModel, 'shakespeare_input.txt', n=2, k=1)
with open("nytimes_article.txt", encoding='utf-8', errors='ignore') as f:
    text = f.read()
m.perplexity(text)

11.018771417967905

In [84]:
# my own text file
# m = create_ngram_model(NgramModel, 'shakespeare_input.txt', n=2, k=1)
# with open("book.txt", encoding='utf-8', errors='ignore') as f:
#     text = f.read()
# m.perplexity(text)

## 2.3 Interpolation [12 points]

The idea of interpolation is to calculate the higher order n-gram probabilities also combining the probabilities for lower-order n-gram models. Like smoothing, this helps us avoid the problem of zeros if we haven't observed the longer sequence in our training data. Here's the math:

$$P_{interpolation}(w_i|w_{i−2} w_{i−1}) = \lambda_1 P(w_i|w_{i−2} w_{i−1}) + \lambda_2 P(w_i|w_{i−1}) + \lambda_3 P(w_i)$$

where $\lambda_1 + \lambda_2 + \lambda_3 = 1$.

We've provided you with another class definition `NgramModelWithInterpolation` that extends `NgramModel` for you to implement interpolation. If you've written your code robustly, you should only need to override the `get_vocab(self)`, `update(self, text)`, and `prob(self, context, char)` methods, along with the initializer.

The value of $n$ passed into `__init__(self, n, k)` is the highest order n-gram to be considered by the model (e.g. $n=2$ will consider 3 different length n-grams). Add-k smoothing should take place only when calculating the individual order n-gram probabilities, not when calculating the overall interpolation probability.

By default set the lambdas to be equal weights, but you should also write a helper function that can be called to overwrite this default. Setting the lambdas in the helper function can either be done heuristically or by using a development set, but in the example code below, we've used the default.

```python
>>> m = NgramModelWithInterpolation(1, 0)
>>> m.update('abab')
>>> m.prob('a', 'a')
0.25
>>> m.prob('a', 'b')
0.75

>>> m = NgramModelWithInterpolation(2, 1)
>>> m.update('abab')
>>> m.update('abcd')
>>> m.prob('~a', 'b')
0.4682539682539682
>>> m.prob('ba', 'b')
0.4349206349206349
>>> m.prob('~c', 'd')
0.27222222222222225
>>> m.prob('bc', 'd')
0.3222222222222222
```
- **Problem 2.3**: implement the `NgramModelWithInterpolation` class [6 points]

In [85]:
class NgramModelWithInterpolation(NgramModel):
    ''' An n-gram model with interpolation '''

    def __init__(self, n, k):
        """Initialize with highest order n and smoothing parameter k"""
        # Initialize equal weights for interpolation
        # Create lower order models (orders 0 to n-1)
        super().__init__(n, k)

        self.models = []
        for i in range(n + 1):
            self.models.append(NgramModel(i, k))


        self.lambdas = []
        for i in range(n + 1):
            self.lambdas.append(1.0 / (n + 1))

    #set custom interpolation weights
    def set_lambdas(self, new_lambdas):
        self.lambdas = new_lambdas


    #ovveride vocab to get all models
    def get_vocab(self):
        vocab = set()
        for model in self.models:
            vocab.update(model.get_vocab())
        return vocab

    #overide to ipdate on all models
    def update(self, text):
        for model in self.models:
            model.update(text)

    #interpolate all models
    def prob(self, context, char):
        total_prob = 0

        # For each model (order 0 to n)
        for i, model in enumerate(self.models):
            # Use appropriate context length for each model
            if len(context) < i:
                # If context is shorter than needed, pad with start symbol
                curr_context = '~' * (i - len(context)) + context
            else:
                # Take last i characters
                curr_context = context[-i:] if i > 0 else ''

            # Get probability from current model and apply lambda weight
            curr_prob = model.prob(curr_context, char)
            total_prob += self.lambdas[i] * curr_prob

        return total_prob

In [86]:
# PennGrader - DO NOT CHANGE
interpolation_test_cases = [
    [(0, 1), ['abab'], ('a', 'a')],
    [(2, 1), ['abab', 'abcd'], ('~c', 'd')],
    [(2, 1), ['abeadab', 'abcd', 'eadb'], ('~a','b')],
    [(2, 1), ['abab', 'abcd', 'aaaaad'], ('bc', 'd')],
]

interpolation_test_results = []
for (n, k), corpus, (context, char) in interpolation_test_cases:
    nm_interpolation = NgramModelWithInterpolation(n, k)
    for text in corpus:
        nm_interpolation.update(text)
    interpolation_test_results.append(nm_interpolation.prob(context, char))

grader.grade(test_case_id = 'test_q23_interpolation', answer = interpolation_test_results)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


- **Answer 2.3**: Experiment with a few different lambdas and values of k and discuss the effects of smoothing and interpolation in terms of perplexity. Alternatively, you can also try different number of n. You may still use text files from 2.2. [6 points]



In [101]:


print("model 1 n=2 k = 1")
m = NgramModelWithInterpolation(2, 1)
m.update('sususususushi')
print(m.prob('~s', 's'))
print(m.prob('~s', 'u'))
print(m.prob('sh', 'i'))
print("model 2 n=2 k = 0.5")
m = NgramModelWithInterpolation(2, 0.5)
m.update('susususushi')
print(m.prob('~~', 's'))
print(m.prob('~s', 'u'))
print(m.prob('~s', 'h'))
print(m.prob('su', 'z'))
print("model 2 with lambdas n=2 k = 0.5")
m = NgramModelWithInterpolation(2, 0.5)
m.set_lambdas([0.1, 0.2, 0.7])
m.update('susususushi')
print(m.prob('~~', 's'))
print(m.prob('~s', 'u'))
print(m.prob('~s', 'h'))
print(m.prob('su', 'z'))
print("model 3 n=7 k = 1")
m = NgramModelWithInterpolation(7, 1)
m.update('susususushi')
print(m.prob('~~~~su','s'))
print(m.prob('~~~~~sh', 'z'))
print(m.prob('~~~susus', 'u'))
print(m.prob('~~~~~~', 's'))
print("model 4 n=7 k = 1")
m = NgramModelWithInterpolation(7, 1)
m.set_lambdas([0.1, 0.2, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1])
m.update('susususushi')
print(m.prob('~~~~su','s'))
print(m.prob('~~~~~sh', 'z'))
print(m.prob('~~susus', 'u'))
print(m.prob('~~~~~~', 's'))


model 1 n=2 k = 1
0.2372549019607843
0.4509803921568627
0.3058823529411765
model 2 n=2 k = 0.5
0.47435897435897434
0.49633699633699635
0.16544566544566544
0.06837606837606837
model 2 with lambdas n=2 k = 0.5
0.4923076923076923
0.5131868131868131
0.17106227106227107
0.07884615384615384
model 3 n=7 k = 1
0.45624999999999993
0.21458333333333335
0.4432539682539682
0.39999999999999997
model 4 n=7 k = 1
0.46750000000000014
0.21666666666666667
0.4601587301587303
0.40000000000000013


In [88]:
m = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n=2, k=1)
m.set_lambdas([0.1, 0.2, 0.7])
print(m.prob('~~t','a'))
print(m.prob('the', 'a'))


0.021487433634107544
0.06109323584027111


In [100]:
m = create_ngram_model(NgramModelWithInterpolation, 'shakespeare_input.txt', n=2, k=1)
m.set_lambdas([0.3, 0.3, 0.4])
print(m.prob('~pe','o'))
print(m.prob('~~e', 'a'))

0.029105724660398395
0.04074692612227151


 I tested multiple ngrams with interpolation. It made sense that a longer length ngram model would have less probabilities since words are not that long. Increasing "k" smoothin would distribute some of these probabilities. The first parts of experiementing were with phrases. I picked repeatable "susu" so I could test the different results for ones that were uncommon such as the  "h" or "i" near the end. I wanted to see how it handles unseen characters in the second model and those look to drop to 0.1 in the bigram model. Changing the lambda values made much smaller differences than the smoothing values, but I can see its use for setting equal weights, or an increasing/decreasing lambda to adjust probabilities.

 I then tested on a provided txt document and found that it was extremely difficult. This is mostl ikely due to the size of the vocabulary and ngrams being unable to figure out the long range dependencies.



# Section 3: Text Classification using N-Grams [20 points + 5 bonus]
**No PennGrader in this section, see `Deliverables` for grading details**
## Overview
Language models can be applied to text classification. If we want to classify a text $D$ into a category $c \in C={c_1, ..., c_N}$. We can pick the category $c$ that has the largest posterior probability given the text. That is,

$$ c^* = arg max_{c \in C} P(c|D) $$

Using Bayes rule, this can be rewritten as:

$$ c^* = arg max_{c \in C} P(D|c) P(c)$$

If we assume that all classes are equally likely, then we can just drop the $P(c)$ term:

$$ = arg max_{c \in C} P(D|c)$$

Here $P(D \mid c)$ is the likelihood of $D$ under category $c$, which can be computed by training language models for all texts associated with category $c$. This technique of text classification is drawn from [literature on authorship identification](http://www.aclweb.org/anthology/E/E03/E03-1053.pdf), where the approach is to learn a separate language model for each author, by training on a data set from that author. Then, to categorize a new text D, they use each language model to calculate the likelihood of D under that model, and pick the  category that assigns the highest probability to D.


## Try it!
We have provided you training and validation datsets consisting of the names of cities. The task is to predict the country a city is in. The following countries are including in the dataset.

```
af	Afghanistan
cn	China
de	Germany
fi	Finland
fr	France
in	India
ir	Iran
pk	Pakistan
za	South Africa
```

We'll set up a leaderboard for the text classification task. **Your job is to configure a set of language models that perform the best on the text classification task.** We will use the `city names` dataset, which you should have already downloaded. The test set has one unlabeled city name per line.

Your code should output a file `test_labels.txt` with one two-letter country code per line.

Feel free to extend the `NgramModel` or `NgramModelWithInterpolation` when creating your language model. Possible ideas to consider and experiment with when creating your model are utilizing a special end-of-text character, trying a new method for determining the vocab, and improving how your model handles novel characters.

## Deliverables
- **Answer 3.1:** Train your best model on the given training data (located in `train` folder), and report accuracy on the given validation data (located in `val` folder) for your final model. In order to receive full credit, your model must be able to outperform all the baselines. [5 points]

- **Answer 3.2:** Describe the parameters you used for the final submission, such as n , k and interpolation. Discuss how you made the decision and why your combination have better results, we encourage using a table to display all validation accuracies of your experiment process. Be sure to include a detailed error analysis: identify several examples of cities on which your final model is making errors, discuss potential reasons. [15 points]

- **Answer 3.3:** Make predictions on the given test data (`cities_test.txt`) and upload `test_labels.txt` to Gradescope, which contains one two-letter country code per line. Please note that testing accuracy should be valid and outperform the baseline in order to receive full credits (included in 3.1). Top 3 will be granted 5 bonus points.

The classifier uses the min perplexity scores to have the most accurate matching possible. I think the flaws to this approach are cities with long names and certain prefixes. Some cities like "Africanda" or prefixes with "San" from af.txt might be mis classified because they can show up more frequently for other countries. While I was tuning parameters, I figured that the 3-gram model seemed to be giving me the best scores of around 0.65. After a little bit of tuning with smoothing, and lambdas I came to a conclusion that equal weights provided the best scores. I believe that the length of an 3 gram model is ideal since some of these city names do have 3 letter suffixes. The 2 gram model would be a little too short while the 4 gram model may be a little too specific after the pre/suffix of these cities. The smoothing factor at 0.7 in combination showed the current best accuracy mostly because of the way the letters are arranged in the names of these cities. While some cities can be predictable to Africana, some cities have names like "dzk" which is very uncommon. Because we have a 3gram model, I assume having equal weight lambdas allows us to capture 3 letter phrases better instead of a partial match or overshooting the phrase. It allows us to handle sequences including "dzk" along with "San".

In [90]:
COUNTRY_CODES = ['af', 'cn', 'de', 'fi', 'fr', 'in', 'ir', 'pk', 'za']

In [91]:
train_dir = 'train'
val_dir = 'val'
test_dir = 'cities_test.txt'

In [92]:
import os

def get_models(train_dir, n, k):
    models = {}
    for filename in os.listdir(train_dir):
        country_code = filename.split('.')[0]  #
        filepath = os.path.join(train_dir, filename)
        models[country_code] = create_ngram_model(NgramModelWithInterpolation, filepath, n, k)

        with open(filepath, encoding='utf-8', errors='ignore') as f:
            for line in f:
                models[country_code].update(line.strip())
    return models

In [ ]:
def train_models(models, val_dir):
    y_pred = []
    y_true = []

    for filename in os.listdir(val_dir):
        filepath = os.path.join(val_dir, filename)
        val_country = filename.split('.')[0]

        with open(filepath, encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.strip()
                perplexities = {}

                for country in models.keys():
                    perplexities[country] = models[country].perplexity(line)
                    pred_country = min(perplexities, key=perplexities.get)
                y_pred.append(pred_country)
                y_true.append(val_country)

    return y_pred, y_true

In [94]:

def accuracy(y_pred, y_true):
    correct = 0
    for i, pred in enumerate(y_pred):
        if pred == y_true[i]:
            correct += 1
    accuracy = correct/len(y_true)
    return accuracy

In [ ]:
def tune_parameters(train_dir, val_dir):
    best_accuracy = 0
    best_params = None
    # for n in [2, 3, 4, 5]:
    #     for k in [0.4, 0.55, 0.7, 1]:
    #         lambda_options = [
    #             [0.4, 0.3, 0.2, 0.1],
    #             [0.25, 0.25, 0.25, 0.25],
    #             [0.1, 0.2, 0.3, 0.4]
    #             [0.1, 0.4, 0.4, 0.1]
    #         ]

    for n in [2, 3]:
        for k in [0.4, 0.5, 0.7, 1]:
            lambda_options = [
                [0.4, 0.3, 0.2, 0.1],
                [0.25, 0.25, 0.25, 0.25],
                [0.1, 0.2, 0.3, 0.4]
            ]

            for lambdas in lambda_options:
                models = get_models(train_dir, n, k)

                for model in models.values():
                    model.set_lambdas(lambdas)

                y_pred, y_true = train_models(models, val_dir)
                current_accuracy = accuracy(y_pred, y_true)

                if current_accuracy > best_accuracy:
                    best_accuracy = current_accuracy
                    best_params = {
                        'n': n,
                        'k': k,
                        'lambdas': lambdas
                    }

                print(f"model ({n}, {k}, {lambdas}): {current_accuracy}")

    return best_params, best_accuracy

In [ ]:

print("model (n, k, [lambdas])")
best_params, best_acc = tune_parameters(train_dir, val_dir)
print("best accuracy:", best_acc)

models = get_models(train_dir, best_params['n'], best_params['k'])
for model in models.values():
    model.set_lambdas(best_params['lambdas'])

y_pred, y_true = train_models(models, val_dir)

model (n, k, [lambdas])
model (2, 0.4, [0.4, 0.3, 0.2, 0.1]): 0.6677777777777778
model (2, 0.4, [0.25, 0.25, 0.25, 0.25]): 0.6677777777777778
model (2, 0.4, [0.1, 0.2, 0.3, 0.4]): 0.6633333333333333
model (2, 0.5, [0.4, 0.3, 0.2, 0.1]): 0.6655555555555556
model (2, 0.5, [0.25, 0.25, 0.25, 0.25]): 0.6655555555555556
model (2, 0.5, [0.1, 0.2, 0.3, 0.4]): 0.6666666666666666
model (2, 0.7, [0.4, 0.3, 0.2, 0.1]): 0.6666666666666666
model (2, 0.7, [0.25, 0.25, 0.25, 0.25]): 0.6644444444444444
model (2, 0.7, [0.1, 0.2, 0.3, 0.4]): 0.6644444444444444
model (2, 1, [0.4, 0.3, 0.2, 0.1]): 0.6666666666666666
model (2, 1, [0.25, 0.25, 0.25, 0.25]): 0.6711111111111111
model (2, 1, [0.1, 0.2, 0.3, 0.4]): 0.6688888888888889
model (3, 0.4, [0.4, 0.3, 0.2, 0.1]): 0.6855555555555556
model (3, 0.4, [0.25, 0.25, 0.25, 0.25]): 0.6933333333333334
model (3, 0.4, [0.1, 0.2, 0.3, 0.4]): 0.7
model (3, 0.5, [0.4, 0.3, 0.2, 0.1]): 0.6844444444444444
model (3, 0.5, [0.25, 0.25, 0.25, 0.25]): 0.6911111111111111
mode

In [97]:
results = []
with open(test_dir, encoding = 'utf-8', errors = 'ignore') as f:
    for line in f:
        perplexities = {}
        for country in models.keys():
            perplexities[country] = models[country].perplexity(line)
        pred_country = min(perplexities, key = perplexities.get)
        results.append(pred_country)

In [98]:
with open('test_labels.txt', 'w') as f:
    for label in results:
        f.write(str(label)+ '\n')

# Submission
### Congratulation on finishing your homework! Here are the deliverables you need to submit to GradeScope
- This notebook and py file: rename to `homework3.ipynb` and `homework3.py`. You can download the notebook and py file by going to the top-left corner of this webpage, `File -> Download -> Download .ipynb/.py`
- `test_labels.txt` from `Section 3`